In [2]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TORCH_SHOW_CPP_STACKTRACES'] = "1"

In [3]:
!pip install torchaudio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
!pip install pydub
!pip install scikit-image

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
import os
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('../lib'))
#import config
import config_DK_AST_1
#import config

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from datetime import datetime
from torchvision import datasets
import librosa
import librosa.display
import os
import skimage.util
import pickle
import math
import collections
import pandas as pd
from sklearn.utils import shuffle
from scipy import signal
import scipy
import matplotlib.pyplot as plt
from scipy.signal import butter , filtfilt
import scipy.io as sio
import scipy.io.wavfile
import copy
from PyTorch import config_pytorch
from PyTorch.runTorch_AST_DK import ASTModel
from evaluate import get_results
from pydub import AudioSegment
import math
import pathlib
import random
import torchaudio
from nnAudio import features
import torchaudio.transforms as T
from torch.utils.data import DataLoader, Dataset, random_split
import time
from sklearn.metrics import accuracy_score
import timm
import warnings
warnings.filterwarnings('ignore')
import gc




/opt/conda/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [6]:
#Source https://betterprogramming.pub/how-to-make-your-pytorch-code-run-faster-93079f3c1f7b
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

## Few Global params

In [7]:
from torch.cuda.amp import *

In [8]:
data_dir = os.path.join(os.path.pardir, 'data', 'audio')
print(data_dir)
plot_dir = os.path.join(os.path.pardir, 'outputs',  'plots')
model_dir = os.path.join(os.path.pardir, 'outputs', 'models') # Model sub-directory created in config_keras or config_pytorch
# this is the batch_size of the dataloader


# set this to True if you want to call the bandpass filter
call_filter = False
# This is the duration of the max. len train file. Due to randomization the length should decrease.
mini_batch_size = 32

../data/audio


In [10]:
import pandas as pd
df = pd.read_csv(config_DK_AST_1.data_df)

# To be kept: please do not edit the test set: these paths select test set A, test set B as described in the paper
idx_test_A = np.logical_and(df['country'] == 'Tanzania', df['location_type'] == 'field')
idx_test_B = np.logical_and(df['country'] == 'UK', df['location_type'] == 'culture')
idx_train = np.logical_not(np.logical_or(idx_test_A, idx_test_B))
df_test_A = df[idx_test_A]
df_test_B = df[idx_test_B]


df_train = df[idx_train]

# Modify by addition or sub-sampling of df_train here


# Assertion to check that train does NOT appear in test:
assert len(np.where(pd.concat([df_train,df_test_A,
                               df_test_B]).duplicated())[0]) == 0, 'Train dataframe contains overlap with Test A, Test B'





In [21]:
def concat_files(df,desired_files = 2 ,resample = False,sample_rate = desired_sr):

    desired_files = desired_files
    running_duration = 0
    #define a datframe to store teh results.
    df_concat_audio = pd.DataFrame(columns = ['filename', 'duration','label','time_start','time_end'])
    # an empty tensor to concatenate file along 0th dimension
    file_tensor = torch.empty(size = (1,0))
    for i in range(desired_files):

        #pick a random integer from 0- len(df)
        idx = np.random.randint(low = 0 , high = len(df))
        print("selecting file at idx = " +str(idx))

        print("shape of initial zero tensor= " + str(file_tensor.shape))
        #select a file at idx = idx
        file_id = df.loc[idx,'id']
        filetag = df.loc[idx,'name']
        _, file_format = os.path.splitext(filetag)
        label = df.loc[idx,'sound_type']
        filename_str = str(file_id) + file_format
        filename = os.path.join(data_dir, str(file_id) + file_format)
        print("^^^^^^^^ Start processing filename = " + str(filename_str) + "^^^^^^^^")
        time_start = running_duration
        print("time_start = " +str(time_start))
        aud_durn = librosa.get_duration(filename = filename)
        print("aud_durn = " + str(aud_durn))
        time_end = running_duration + aud_durn
        print("time_end = " +str(time_end))
        running_duration = time_end
        # add values to the data frame
        df_concat_audio.loc[i] = [filename_str , aud_durn , label,time_start,time_end] 

        #now get the features.
        signal_8k, rate = torchaudio.load(filename , normalize = True)
        if resample == True :
            transform = T.Resample(orig_freq = 8000,new_freq = 16000)
            signal_file = transform(signal_8k)
        else:
            signal_file = signal_8k
        print("shape of signal file = " + str(signal_file.shape))
        #now concatenate tensor
        file_tensor = torch.cat((file_tensor,signal_file),dim = 1)
        print("shape post concatenate = " + str(file_tensor.shape))
    
        
    return(file_tensor , df_concat_audio)
    
    
    

# pick 

In [29]:
#pick one from 8k or 44100
desired_sr = 8000
#select the values from the original df with the desired freq.
temp_df = df.loc[(df['sample_rate'] == desired_sr)]
filtered_df = temp_df.reset_index(drop=True) 



file_tensor , df_concat_audio = concat_files(filtered_df,desired_files = 20 ,resample = False, sample_rate = desired_sr)
torchaudio.save("test_concat_file.wav" ,file_tensor,sample_rate= desired_sr )

selecting file at idx = 2804
shape of initial zero tensor= torch.Size([1, 0])
^^^^^^^^ Start processing filename = 208381.wav^^^^^^^^
time_start = 0
aud_durn = 2.56
time_end = 2.56
shape of signal file = torch.Size([1, 20480])
shape post concatenate = tensor([[ 6.1035e-05,  6.4087e-04, -7.3242e-04,  ...,  2.1362e-04,
          7.9346e-04,  3.0518e-04]])
selecting file at idx = 478
shape of initial zero tensor= torch.Size([1, 20480])
^^^^^^^^ Start processing filename = 531.wav^^^^^^^^
time_start = 2.56
aud_durn = 0.10575
time_end = 2.66575
shape of signal file = torch.Size([1, 846])
shape post concatenate = tensor([[ 6.1035e-05,  6.4087e-04, -7.3242e-04,  ..., -4.7302e-03,
         -1.4343e-03,  3.1738e-03]])
selecting file at idx = 398
shape of initial zero tensor= torch.Size([1, 21326])
^^^^^^^^ Start processing filename = 475.wav^^^^^^^^
time_start = 2.66575
aud_durn = 0.20875
time_end = 2.8745000000000003
shape of signal file = torch.Size([1, 1670])
shape post concatenate = tensor(

In [30]:

print(df_concat_audio)

      filename   duration       label  time_start    time_end
0   208381.wav   2.560000    mosquito           0    2.560000
1      531.wav   0.105750    mosquito        2.56    2.665750
2      475.wav   0.208750    mosquito     2.66575    2.874500
3      197.wav   0.203250    mosquito      2.8745    3.077750
4   207237.wav   2.560000    mosquito     3.07775    5.637750
5   219899.wav  54.272000  background     5.63775   59.909750
6      535.wav   0.124375    mosquito    59.90975   60.034125
7      282.wav   0.054750    mosquito   60.034125   60.088875
8   199889.wav  13.800000    mosquito   60.088875   73.888875
9   207213.wav   2.559875    mosquito   73.888875   76.448750
10  208642.wav   5.120000    mosquito    76.44875   81.568750
11  206955.wav   2.560000    mosquito    81.56875   84.128750
12  219734.wav  54.272000  background    84.12875  138.400750
13  199941.wav   9.700000    mosquito   138.40075  148.100750
14  207743.wav   2.559875    mosquito   148.10075  150.660625
15  2193

,filename,duration,label,time_start,time_end


,id,length,name,sample_rate,record_datetime,sound_type,species,gender,fed,plurality,age,method,mic_type,device_type,country,district,province,place,location_type
0,199980,1121.880000,background.wav,8000,08-09-16 08:00,background,NaN,NaN,NaN,NaN,NaN,NaN,phone,Alcatel 4009X,USA,Georgia,Atlanta,"CDC insect cultures, Atlanta",culture
1,53,0.463456,CDC_Ae-aegypti_labelled_800.wav,8000,08-09-16 08:00,mosquito,ae aegypti,NaN,NaN,Single,NaN,NaN,phone,Alcatel 4009X,USA,Georgia,Atlanta,"CDC insect cultures, Atlanta",culture
2,57,0.170249,CDC_Ae-aegypti_labelled_800.wav,8000,08-09-16 08:00,mosquito,ae aegypti,NaN,NaN,Single,NaN,NaN,phone,Alcatel 4009X,USA,Georgia,Atlanta,"CDC insect cultures, Atlanta",culture
3,61,0.104041,CDC_Ae-aegypti_labelled_800.wav,8000,08-09-16 08:00,mosquito,ae aegypti,NaN,NaN,Single,NaN,NaN,phone,Alcatel 4009X,USA,Georgia,Atlanta,"CDC insect cultures, Atlanta",culture
4,69,0.274290,CDC_Ae-aegypti_labelled_800.wav,8000,08-09-16 08:00,mosquito,ae aegypti,NaN,NaN,Single,NaN,NaN,phone,Alcatel 4009X,USA,Georgia,Atlanta,"CDC insect cultures, Atlanta",culture
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7085,208655,2.560000,2020-11-26_22.00.13.793.wav,8000,26-11-20 22:00,mosquito,an arabiensis,NaN,NaN,NaN,NaN,HBN,phone,itel A16,Tanzania,Kilombero District,Morogoro,Ifakara,field
7086,208623,2.560000,2020-11-26_22.00.13.793.wav,8000,26-11-20 22:00,mosquito,an arabiensis,NaN,NaN,NaN,NaN,HBN,phone,itel A16,Tanzania,Kilombero District,Morogoro,Ifakara,field
7087,208713,2.560000,2020-11-27_04.30.13.788.wav,8000,27-11-20 04:30,mosquito,an arabiensis,NaN,NaN,NaN,NaN,HBN,phone,itel A16,Tanzania,Kilombero District,Morogoro,Ifakara,field
7088,208712,2.560000,2020-11-27_04.30.13.788.wav,8000,27-11-20 04:30,mosquito,an arabiensis,NaN,NaN,NaN,NaN,HBN,phone,itel A16,Tanzania,Kilombero District,Morogoro,Ifakara,field
